LTIM time series, 1991 to 2016 Age and Sex

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
from pathlib import Path

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

%run scrape_ons.ipynb

metadata = scrape('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/' \
                  'internationalmigration/datasets/longterminternationalmigrationageandsextable207')
metadata

{'about': 'Age and sex of migrants entering or leaving UK. Estimates of Long-Term International Migration, annual table.',
 'fileURL': 'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/longterminternationalmigrationageandsextable207/current/2.07ltimageandsex1991to2016.xls',
 'mailto': 'mailto:migstatsunit@ons.gsi.gov.uk',
 'releaseDate': datetime.date(2017, 11, 30),
 'title': 'Long-term international migration 2.07, age and sex, UK and England and Wales'}

https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/longterminternationalmigrationageandsextable207/current/2.07ltimageandsex1991to2016.xls

In [3]:
inputFile = sourceFolder / 'data.xls'
response = session.get(metadata['fileURL'])
with open(inputFile, 'wb') as f:
  f.write(response.content)
tab = loadxlstabs(inputFile, sheetids='Table 2.07')[0]

Loading in\data.xls which has size 216576 bytes
Table names: ['Table 2.07']


In [4]:
observations = tab.excel_ref("B15").expand(RIGHT).filter(contains_string("Estimate")).expand(DOWN).is_not_blank().is_not_whitespace()
observations

{<AO41 6.0>, <AH117 7.0>, <R84 26.0>, <AF146 -12.0>, <AK94 -19.0>, <Y42 116.0>, <AF194 'No'>, <P26 115.0>, <R45 103.0>, <AK133 9.0>, <M162 -9.0>, <R155 -34.0>, <P80 12.0>, <AF123 17.0>, <T159 -37.0>, <P67 -104.0>, <AO163 -4.0>, <R74 -46.0>, <K45 23.0>, <Y71 -99.0>, <AF117 10.0>, <AM34 3.0>, <F82 35.0>, <B86 140.0>, <B127 528.0>, <K46 22.0>, <M177 3.0>, <AM117 3.0>, <F128 233.0>, <AF75 -20.0>, <W179 74.0>, <AH181 -9.0>, <AM192 0.0>, <P33 208.0>, <Y184 62.0>, <AF51 -15.0>, <M168 5.0>, <AH164 -11.0>, <R170 8.0>, <AH175 0.0>, <AD130 37.0>, <P23 94.0>, <AR92 245.0>, <I76 'No'>, <D134 299.0>, <AA183 34.0>, <P145 -76.0>, <AM50 -4.0>, <W101 45.0>, <B40 591.0>, <AA65 -84.0>, <W155 -163.0>, <AH56 -7.0>, <AO83 -3.0>, <Y58 -79.0>, <D118 234.0>, <D61 -196.0>, <W42 218.0>, <AA74 -67.0>, <AK124 6.0>, <B168 51.0>, <K52 -23.0>, <B83 76.0>, <M129 11.0>, <B135 540.0>, <D191 138.0>, <T186 52.0>, <P161 -79.0>, <R111 32.0>, <D69 -185.0>, <AF41 23.0>, <T168 27.0>, <AA97 27.0>, <F67 -206.0>, <AH173 2.0>, <W12

In [5]:
CI = observations.shift(RIGHT)
CI

{<AE116 5.0>, <G119 22.0>, <AN45 2.0>, <U56 9.0>, <AL47 ''>, <AE130 9.0>, <N54 5.0>, <U93 17.0>, <AN64 6.0>, <U66 7.0>, <Q41 17.0>, <AP85 4.0>, <G52 14.0>, <J42 5.0>, <X127 27.0>, <N59 5.0>, <J134 6.0>, <Q81 15.0>, <AP177 3.0>, <L156 3.0>, <Z145 11.0>, <AI127 6.0>, <X124 24.0>, <AE29 6.0>, <U37 17.0>, <L182 8.0>, <AL27 4.0>, <AN175 3.0>, <AN131 4.0>, <J149 6.0>, <AL136 ''>, <AB95 22.0>, <G96 42.0>, <C121 31.0>, <L134 5.0>, <AL46 4.0>, <X181 33.0>, <AI56 3.0>, <AB112 7.0>, <AN126 3.0>, <E133 24.0>, <AL158 2.0>, <AG175 5.0>, <J39 9.0>, <E59 21.0>, <J174 13.0>, <N31 5.0>, <X128 19.0>, <S121 15.0>, <G146 15.0>, <C128 28.0>, <AL99 7.0>, <Z158 11.0>, <AP98 3.0>, <AN75 3.0>, <J140 7.0>, <Z40 15.0>, <AL141 2.0>, <AI61 9.0>, <E98 28.0>, <N102 6.0>, <AG133 10.0>, <Q80 15.0>, <AG168 6.0>, <L34 8.0>, <Z150 13.0>, <AI86 5.0>, <AB53 9.0>, <S143 7.0>, <AG89 12.0>, <AB103 18.0>, <E26 17.0>, <X64 21.0>, <J15 '+/-CI'>, <G127 24.0>, <L64 6.0>, <J139 8.0>, <AI129 4.0>, <AB188 14.0>, <N192 6.0>, <L54 5.0>,

In [6]:
Value_Revision = tab.excel_ref("AR15").filter(contains_string("Estimate")).expand(DOWN).is_not_blank().is_not_whitespace()
Value_Revision

{<AR92 245.0>, <AR15 'Estimate'>, <AR90 153.0>, <AR96 163.0>, <AR99 215.0>, <AR89 171.0>, <AR95 233.0>, <AR98 252.0>, <AR94 198.0>, <AR93 206.0>, <AR97 198.0>, <AR91 148.0>}

In [7]:
CI_Revision = Value_Revision.shift(RIGHT)
CI_Revision

{<AS98 37.0>, <AS15 '+/-CI'>, <AS99 35.0>, <AS92 49.0>, <AS97 37.0>, <AS89 40.0>, <AS95 48.0>, <AS96 57.0>, <AS90 43.0>, <AS94 52.0>, <AS93 49.0>, <AS91 46.0>}

In [8]:
Sex = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
Sex

{<AK13 'Persons'>, <B13 'Persons'>, <K13 'Males'>, <AH13 'Females'>, <P13 'Persons'>, <W13 'Persons'>, <AF13 'Males'>, <I13 'Persons'>, <AA13 'Females'>, <F13 'Females'>, <M13 'Females'>, <R13 'Males'>, <AO13 'Females'>, <Y13 'Males'>, <T13 'Females'>, <AR13 'Persons'>, <AD13 'Persons'>, <AM13 'Males'>, <D13 'Males'>}

In [9]:
Age = tab.excel_ref('B12').expand(RIGHT).is_not_blank()
Age

{<AR12 'All Ages'>, <B12 'All ages'>, <P12 '15-24'>, <I12 'Under 15'>, <W12 '25-44'>, <AD12 '45-59/642'>, <AK12 '60/65 and over3'>}

In [10]:
Revision = tab.excel_ref('B14').expand(RIGHT).is_not_blank()
Revision

{<B14 '2011 Census Revisions1'>, <AR14 'Original Estimates1'>}

In [11]:
Flow = tab.excel_ref('A').expand(DOWN).by_index([19,48,77,108,137,166])
Flow

{<A137 'Outflow'>, <A166 'Balance'>, <A77 'Balance'>, <A108 'Inflow'>, <A19 'Inflow'>, <A48 'Outflow'>}

In [12]:

Geography = tab.excel_ref('A').expand(DOWN).by_index([17,106])
Geography

{<A106 'England and Wales'>, <A17 'United Kingdom'>}

In [13]:
Year = tab.excel_ref('A19').expand(DOWN) - Flow - tab.excel_ref('A').expand(DOWN).by_index([47,76,105,136,165,194,106]) - tab.excel_ref('A195').expand(DOWN)  
Year

{<A94 2006.0>, <A147 1999.0>, <A146 1998.0>, <A117 1998.0>, <A112 1993.0>, <A51 1992.0>, <A42 2012.0>, <A32 2002.0>, <A189 2012.0>, <A21 1991.0>, <A50 1991.0>, <A86 1998.0>, <A57 1998.0>, <A125 2006.0>, <A183 2006.0>, <A71 2012.0>, <A110 1991.0>, <A182 2005.0>, <A49 ''>, <A185 2008.0>, <A163 '2015'>, <A28 1998.0>, <A141 1993.0>, <A31 2001.0>, <A138 ''>, <A126 2007.0>, <A129 2010.0>, <A46 '2016'>, <A155 2007.0>, <A104 '2016'>, <A193 '2016'>, <A79 1991.0>, <A44 2014.0>, <A80 1992.0>, <A23 1993.0>, <A39 2009.0>, <A91 2003.0>, <A177 2000.0>, <A181 2004.0>, <A36 2006.0>, <A132 2013.0>, <A161 2013.0>, <A191 2014.0>, <A70 2011.0>, <A145 1997.0>, <A180 2003.0>, <A168 1991.0>, <A186 2009.0>, <A124 2005.0>, <A135 '2016'>, <A67 2008.0>, <A38 2008.0>, <A142 1994.0>, <A111 1992.0>, <A66 2007.0>, <A152 2004.0>, <A171 1994.0>, <A30 2000.0>, <A93 2005.0>, <A24 1994.0>, <A82 1994.0>, <A176 1999.0>, <A178 2001.0>, <A116 1997.0>, <A187 2010.0>, <A53 1994.0>, <A26 1996.0>, <A69 2010.0>, <A75 '2016'>, <A11

In [14]:
Tag = tab.excel_ref('B15').expand(RIGHT).is_not_blank()
Tag

{<P15 'Estimate'>, <F15 'Estimate'>, <AS15 '+/-CI'>, <C15 '+/-CI'>, <AR15 'Estimate'>, <AH15 'Estimate'>, <AD15 'Estimate'>, <Q15 '+/-CI'>, <K15 'Estimate'>, <AP15 '+/-CI'>, <L15 '+/-CI'>, <Y15 'Estimate'>, <AI15 '+/-CI'>, <B15 'Estimate'>, <Z15 '+/-CI'>, <AF15 'Estimate'>, <S15 '+/-CI'>, <AE15 '+/-CI'>, <E15 '+/-CI'>, <W15 'Estimate'>, <M15 'Estimate'>, <AL15 '+/-CI'>, <N15 '+/-CI'>, <AK15 'Estimate'>, <AA15 'Estimate'>, <D15 'Estimate'>, <AB15 '+/-CI'>, <AO15 'Estimate'>, <R15 'Estimate'>, <G15 '+/-CI'>, <AN15 '+/-CI'>, <U15 '+/-CI'>, <AM15 'Estimate'>, <I15 'Estimate'>, <X15 '+/-CI'>, <J15 '+/-CI'>, <T15 'Estimate'>, <AG15 '+/-CI'>}

In [15]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,LEFT),
            HDim(Geography,'Geography', CLOSEST, ABOVE),
            HDim(Age,'Age',CLOSEST,LEFT),
            HDim(Sex,'Sex',CLOSEST,LEFT),
            HDim(Flow,'Flow',CLOSEST,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People (thousands)'),
            HDim(CI,'CI',DIRECTLY,RIGHT),
            HDim(Revision,'Revision',DIRECTLY,ABOVE),
            HDim(CI_Revision,'CI_Revision',DIRECTLY,RIGHT),
            HDim(Value_Revision,'Value_Revision',DIRECTLY,RIGHT)
            ]

In [16]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
savepreviewhtml(c1)

In [17]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Year,Geography,Age,Sex,Flow,Measure Type,Unit,CI,Revision,CI_Revision,Value_Revision
0,,Estimate,None,None,All ages,Persons,None,Count,People (thousands),+/-CI,2011 Census Revisions1,+/-CI,Estimate
1,,Estimate,None,None,All ages,Males,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
2,,Estimate,None,None,All ages,Females,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
3,,Estimate,None,None,Under 15,Persons,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
4,,Estimate,None,None,Under 15,Males,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
5,,Estimate,None,None,Under 15,Females,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
6,,Estimate,None,None,15-24,Persons,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
7,,Estimate,None,None,15-24,Males,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
8,,Estimate,None,None,15-24,Females,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate
9,,Estimate,None,None,25-44,Persons,None,Count,People (thousands),+/-CI,None,+/-CI,Estimate


In [18]:
new_table.shape

(2947, 13)

In [19]:
new_table.count()

OBS               2947
DATAMARKER         128
Year              2819
Geography         2928
Age               2947
Sex               2947
Flow              2928
Measure Type      2947
Unit              2947
CI                2947
Revision           175
CI_Revision        228
Value_Revision     228
dtype: int64

In [20]:
new_table = new_table[new_table['Year'].isnull() == False]

In [21]:
new_table.dtypes

OBS               object
DATAMARKER        object
Year              object
Geography         object
Age               object
Sex               object
Flow              object
Measure Type      object
Unit              object
CI                object
Revision          object
CI_Revision       object
Value_Revision    object
dtype: object

In [22]:
# def user_perc2(x,y):
    
#     if (str(x) ==  ':') & (str(y) == '2011 Census Revisions1') : 
        
#         return '2011 Census Revisions'
#     else:
#         return ''
    
# new_table['Revision'] = new_table.apply(lambda row: user_perc2(row['CI'], row['Revision']), axis = 1)

In [23]:
# def user_perc2(x,y):
    
#     if ((str(x) ==  ':') & (str(y) == '2011 Census Revisions1')) | ((str(y) == 'Original Estimates1')): 
        
#         return str(y)
#     else:
#         return None
    
# new_table['temp'] = new_table.apply(lambda row: user_perc2(row['CI'], row['Revision']), axis = 1)

In [24]:
new_table.head()

,OBS,DATAMARKER,Year,Geography,Age,Sex,Flow,Measure Type,Unit,CI,Revision,CI_Revision,Value_Revision
19,329,NaN,1991.0,United Kingdom,All ages,Persons,Inflow,Count,People (thousands),23.0,2011 Census Revisions1,None,None
20,157,NaN,1991.0,United Kingdom,All ages,Males,Inflow,Count,People (thousands),15.0,None,None,None
21,172,NaN,1991.0,United Kingdom,All ages,Females,Inflow,Count,People (thousands),18.0,None,None,None
22,53,NaN,1991.0,United Kingdom,Under 15,Persons,Inflow,Count,People (thousands),10.0,None,None,None
23,23,NaN,1991.0,United Kingdom,Under 15,Males,Inflow,Count,People (thousands),6.0,None,None,None


In [25]:
# new_table[new_table['DATAMARKER'] != 'None']

In [26]:
# new_table.OBS.fillna('0', inplace = True)

In [27]:
# new_table.fillna('None', inplace = True)

In [28]:
# def user_perc(x,y):
    
#     if str(x) == 'None':
#         return y
#     else:
#         return 0
    
# new_table['OBS'] = new_table.apply(lambda row: user_perc(row['DATAMARKER'], row['OBS']), axis = 1)


In [29]:
new_table.count()

OBS               2819
DATAMARKER           0
Year              2819
Geography         2819
Age               2819
Sex               2819
Flow              2819
Measure Type      2819
Unit              2819
CI                2819
Revision           167
CI_Revision        209
Value_Revision     209
dtype: int64

In [30]:
def user_perc1(x,y,z):
    
    if (str(x) ==  ':') & (str(y) == '2011 Census Revisions1') : 
        
        return z
    else:
        return None
    
new_table['Value_Revision'] = new_table.apply(lambda row: user_perc1(row['CI'], row['Revision'],row['Value_Revision']), axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
def user_perc2(x,y,z):
    
    if (str(x) ==  ':') & (str(y) == '2011 Census Revisions1') : 
        
        return z
    else:
        return None
    
new_table['CI_Revision'] = new_table.apply(lambda row: user_perc1(row['CI'], row['Revision'],row['CI_Revision']), axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
# new_table.to_csv('abc.csv')

In [33]:
new_table.count()

OBS               2819
DATAMARKER           0
Year              2819
Geography         2819
Age               2819
Sex               2819
Flow              2819
Measure Type      2819
Unit              2819
CI                2819
Revision           167
CI_Revision         11
Value_Revision      11
dtype: int64

In [34]:
new_table = new_table[new_table['Revision'] != 'Original Estimates1']

In [35]:
new_table.count()

OBS               2808
DATAMARKER           0
Year              2808
Geography         2808
Age               2808
Sex               2808
Flow              2808
Measure Type      2808
Unit              2808
CI                2808
Revision           156
CI_Revision         11
Value_Revision      11
dtype: int64

In [36]:
new_table.head(5)

,OBS,DATAMARKER,Year,Geography,Age,Sex,Flow,Measure Type,Unit,CI,Revision,CI_Revision,Value_Revision
19,329,NaN,1991.0,United Kingdom,All ages,Persons,Inflow,Count,People (thousands),23.0,2011 Census Revisions1,None,None
20,157,NaN,1991.0,United Kingdom,All ages,Males,Inflow,Count,People (thousands),15.0,None,None,None
21,172,NaN,1991.0,United Kingdom,All ages,Females,Inflow,Count,People (thousands),18.0,None,None,None
22,53,NaN,1991.0,United Kingdom,Under 15,Persons,Inflow,Count,People (thousands),10.0,None,None,None
23,23,NaN,1991.0,United Kingdom,Under 15,Males,Inflow,Count,People (thousands),6.0,None,None,None


In [37]:
new_table['Revision'] = new_table['Revision'].str.rstrip('1')

In [38]:
new_table['Age'] = new_table['Age'].str.rstrip('23')

In [39]:
new_table['Value'] = new_table['OBS'].astype(int)

In [40]:
new_table['Year'] = new_table['Year'].astype(str)

In [41]:
new_table['Year'] = pd.to_numeric(new_table['Year'])

In [42]:
new_table['Year'] = new_table['Year'].astype(int)

In [43]:
# new_table['Year'] = new_table['Year'].astype(str)
# new_table['Year'] = pd.to_numeric(new_table['Year'])
# new_table['Year'] = new_table['Year'].astype(int)

In [44]:
# def user_perc1(x,y):
    
#     if str(x) == 'All ages':
#         return x  + ':' +  y
#     else:
#         return x
    
# new_table['Age'] = new_table.apply(lambda row: user_perc1(row['Age'], row['Revision']), axis = 1)

In [45]:
new_table['Geography'] = new_table['Geography'].map(lambda cell:cell.replace('United Kingdom', 'K02000001'))

In [46]:
new_table['Geography'] = new_table['Geography'].map(lambda cell:cell.replace('England and Wales', 'K04000001'))

In [47]:
new_table.dtypes

OBS               object
DATAMARKER        object
Year               int32
Geography         object
Age               object
Sex               object
Flow              object
Measure Type      object
Unit              object
CI                object
Revision          object
CI_Revision       object
Value_Revision    object
Value              int32
dtype: object

In [48]:
new_table['CI'] = pd.to_numeric(new_table['CI'], errors='coerce').fillna(0)

In [49]:
new_table['CI'] = new_table['CI'].astype(int)

In [50]:
# new_table['Value_Revision'] = new_table['Value_Revision'].astype(str)

In [51]:
# new_table['Value_Revision'] = pd.to_numeric(new_table['Value_Revision'])

In [52]:
# new_table['Value_Revision'] = new_table['Value_Revision'].astype(int)

In [53]:
# new_table['Value_Revision'] = pd.to_numeric(new_table['Value_Revision'], errors='coerce')

In [54]:
# new_table['Value_Revision'] = new_table['Value_Revision'].astype(int)

In [55]:
# new_table['CI_Revision'] = pd.to_numeric(new_table['CI_y'], errors='coerce').fillna(0)

In [56]:
# new_table['CI_Revision'] = new_table['CI_Revision'].astype(str)

In [57]:
# new_table['CI_Revision'] = pd.to_numeric(new_table['CI_Revision'])

In [58]:
# new_table['CI_Revision'] = new_table['CI_Revision'].astype(int)

In [59]:
# def user_perc3(x):
    
#     if str(x) == '0':
#         return ''
#     else:
#         return x
    
# new_table['Value_Revision'] = new_table.apply(lambda row: user_perc3(row['Value_Revision']), axis = 1)

In [60]:
# def user_perc4(x):
    
#     if str(x) == '0':
#         return ''
#     else:
#         return x
    
# new_table['CI_Revision'] = new_table.apply(lambda row: user_perc4(row['CI_Revision']), axis = 1)

In [61]:
new_table = new_table[['Geography','Year','Age', 'Sex', 'Flow','Measure Type','Value','CI','Unit','Value_Revision','CI_Revision']]

In [62]:
new_table.dtypes

Geography         object
Year               int32
Age               object
Sex               object
Flow              object
Measure Type      object
Value              int32
CI                 int32
Unit              object
Value_Revision    object
CI_Revision       object
dtype: object

In [63]:
new_table.head(5)

,Geography,Year,Age,Sex,Flow,Measure Type,Value,CI,Unit,Value_Revision,CI_Revision
19,K02000001,1991,All ages,Persons,Inflow,Count,329,23,People (thousands),None,None
20,K02000001,1991,All ages,Males,Inflow,Count,157,15,People (thousands),None,None
21,K02000001,1991,All ages,Females,Inflow,Count,172,18,People (thousands),None,None
22,K02000001,1991,Under 15,Persons,Inflow,Count,53,10,People (thousands),None,None
23,K02000001,1991,Under 15,Males,Inflow,Count,23,6,People (thousands),None,None


In [64]:
new_table.tail(5)

,Geography,Year,Age,Sex,Flow,Measure Type,Value,CI,Unit,Value_Revision,CI_Revision
2923,K04000001,2016,45-59/64,Males,Balance,Count,-1,7,People (thousands),None,None
2924,K04000001,2016,45-59/64,Females,Balance,Count,4,7,People (thousands),None,None
2925,K04000001,2016,60/65 and over,Persons,Balance,Count,-5,6,People (thousands),None,None
2926,K04000001,2016,60/65 and over,Males,Balance,Count,-2,5,People (thousands),None,None
2927,K04000001,2016,60/65 and over,Females,Balance,Count,-3,3,People (thousands),None,None


In [65]:
new_table.count()

Geography         2808
Year              2808
Age               2808
Sex               2808
Flow              2808
Measure Type      2808
Value             2808
CI                2808
Unit              2808
Value_Revision      11
CI_Revision         11
dtype: int64

In [66]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('tidydata2_7.csv'), index = False)

In [67]:
writeMetadata(metadata, 'ONS-LTIM-Reason-for-Migration')